<a href="https://colab.research.google.com/github/yankit247/Dashtoon_code/blob/main/Dashtoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# Import necessary libraries
import pandas as pd
import numpy as np
from keras import backend as K
from keras.preprocessing.image import load_img, save_img, img_to_array
import matplotlib.pyplot as plt
from keras.applications import vgg19
from keras.models import Model
from scipy.optimize import fmin_l_bfgs_b
import tensorflow as tf

# Update paths for Google Drive
StylePath = '/content/drive/MyDrive/styled_images/'
ContentPath = '/content/drive/MyDrive/content_images/'

In [ ]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()
# Function to run style transfer
def Run_StyleTransfer(base_image_path, style_image_path, save_path):
    # Placeholder values, replace with your actual values
    img_nrows = 400
    img_ncols = 400

    base_image = preprocess_image(base_image_path, img_nrows, img_ncols)
    style_reference_image = preprocess_image(style_image_path, img_nrows, img_ncols)

    if K.image_data_format() == 'channels_first':
        combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
    else:
        combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

    input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)

    # Load VGG19 model
    vgg19_weights = '/content/drive/MyDrive/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model = vgg19.VGG19(input_tensor=input_tensor, include_top=False, weights=vgg19_weights)
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

    content_weight = 0.025
    style_weight = 1.0

    # Extract features from the model
    content_layer = 'block5_conv2'
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    base_image_features = outputs_dict[content_layer][0, :, :, :]
    combination_features = outputs_dict[content_layer][2, :, :, :]

    # Calculate content loss
    content_score = get_content_loss(base_image_features, combination_features)

   # Function to calculate style loss
def get_style_loss(style, combination, img_nrows, img_ncols):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# Function to run style transfer
def Run_StyleTransfer(base_image_path, style_image_path, save_path):
    # Placeholder values, replace with your actual values
    img_nrows = 400
    img_ncols = 400

    base_image = preprocess_image(base_image_path, img_nrows, img_ncols)
    style_reference_image = preprocess_image(style_image_path, img_nrows, img_ncols)

    if K.image_data_format() == 'channels_first':
        combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
    else:
        combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

    input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)

    # Load VGG19 model
    vgg19_weights = '/content/drive/MyDrive/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    model = vgg19.VGG19(input_tensor=input_tensor, include_top=False, weights=vgg19_weights)
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

    content_weight = 0.025
    style_weight = 1.0

    # Extract features from the model
    content_layer = 'block5_conv2'
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    base_image_features = outputs_dict[content_layer][0, :, :, :]
    combination_features = outputs_dict[content_layer][2, :, :, :]

    # Calculate content loss
    content_score = get_content_loss(base_image_features, combination_features)

    # Calculate style loss
    style_score = 0
    for layer_name in style_layers:
        style_reference_features = outputs_dict[layer_name][1, :, :, :]
        combination_features = outputs_dict[layer_name][2, :, :, :]
        style_score += get_style_loss(style_reference_features, combination_features, img_nrows, img_ncols)

    # Combine content and style scores to get the total loss
    loss = content_weight * content_score + style_weight * style_score

    # Get the gradients of the generated image wrt the loss
    grads = K.gradients(loss, combination_image)

    # Combine loss and gradients
    outputs = [loss]
    if isinstance(grads, (list, tuple)):
        outputs += grads
    else:
        outputs.append(grads)

    # Create a function to calculate loss and gradients
    f_outputs = K.function([combination_image], outputs)

    # Define an evaluator class
    class Evaluator(object):
        def __init__(self):
            self.loss_value = None
            self.grads_values = None

        def loss(self, x):
            assert self.loss_value is None
            x = x.reshape((1, img_nrows, img_ncols, 3))
            outs = f_outputs([x])
            loss_value = outs[0]
            grad_values = np.array(outs[1:]).flatten().astype('float64')
            self.loss_value = loss_value
            self.grad_values = grad_values
            return self.loss_value

        def grads(self, x):
            assert self.loss_value is not None
            grad_values = np.copy(self.grad_values)
            self.loss_value = None
            self.grad_values = None
            return grad_values

    # Initialize evaluator
    evaluator = Evaluator()

    # Run L-BFGS optimization
    iterations = 200
    best_loss, best_img = float('inf'), None

    # Initialize x_opt
    x_opt = preprocess_image(base_image_path, img_nrows, img_ncols)

    for i in range(iterations):
        x_opt, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                             x_opt.flatten(),
                                             fprime=evaluator.grads,
                                             maxfun=20,
                                             disp=True,
                                            )
        if min_val < best_loss:
            best_loss = min_val
            best_img = x_opt.copy()

    # Save the final image to Google Drive
    save_path = '/content/drive/MyDrive/' + save_path
    imgx = deprocess_image(best_img.copy())
    save_img(save_path, imgx)

    return imgx, save_path

    # Save the final image to Google Drive
    save_path = '/content/drive/MyDrive/' + save_path
    imgx = deprocess_image(best_img.copy())
    save_img(save_path, imgx)

    return imgx, save_path

# Example usage for the first set of images
base_image_path_1 = '/content/drive/MyDrive/content_images/Andrei_Rublev_14.jpg'
style_image_path_1 = '/content/drive/MyDrive/styled_images/0.jpg'
imgg, save_path_1 = Run_StyleTransfer(base_image_path_1, style_image_path_1, 'final_image_1.jpg')

# Display the final image
plt.title("Final Image", fontsize=20)
plt.imshow(imgg)
plt.show()
